# Bayesian A/B Test

Autor: Luca Honegger, lh@sinnhaft.ch

Quelle: Dies ist eine Python-Portierung der UXtoolbox in R von Mohsen Rafiei, https://github.com/mohsen-rafiei/UXtoolbox

## 🧠 Was macht diese Funktion?

Die Funktion `bayesian_ab_test()` führt einen **Bayesschen A/B-Test** durch.
Sie vergleicht zwei Gruppen (z. B. Variante A vs. Variante B) basierend auf Erfolgen und Gesamtversuchen.

### 🔍 Was wird berechnet?

* Wie wahrscheinlich ist es, dass **A besser abschneidet als B** – und umgekehrt
* Wie hoch ist die **erwartete Erfolgsrate** für A und B
* Wie gross ist der **wahrscheinliche Unterschied** (Credible Interval)

Dabei wird nicht nur ein einfacher Mittelwert verglichen, sondern **Unsicherheit mit einbezogen** – das ist der Vorteil von Bayesian Statistics.

## 🧪 Beispiel aus dem Business-Alltag

Stell dir vor, du testest zwei Versionen einer Landingpage:

* **Variante A:** 65 Conversions bei 90 Besuchern
* **Variante B:** 72 Conversions bei 110 Besuchern (z. B. aus einer kleinen gezielten Kampagne)

Mit `bayesian_ab_test([65, 72], [90, 100])` bekommst du:

* Wie wahrscheinlich es ist, dass **Variante A oder B besser ist**
* Und wie gross der Unterschied **realistisch** sein kann

Das hilft dir bei **datenbasierten Entscheidungen**, auch wenn die Stichprobe klein oder ungleich verteilt ist – typisches Szenario z. B. bei MVP-Tests, Marketing-Kampagnen oder Produktexperimenten.


## `bayesian_ab_test`
mit SciPy

In [2]:
import numpy as np
from scipy.stats import beta

def bayesian_ab_test(successes, trials, ci=0.95, n_samples=10000):
    """
    Führt einen bayesschen A/B-Test durch mit Beta-Verteilungen als Posterior.

    Parameter:
    - successes: Liste mit Erfolgen für A und B (z. B. [10, 20])
    - trials: Liste mit Versuchen für A und B (z. B. [800, 25])
    - n_samples: Anzahl der Monte-Carlo-Samples
    - ci: Konfidenzintervall (Credible Interval), z. B. 0.95 für 95 %

    Rückgabe:
    - Dictionary mit:
        - mittlere Erfolgswahrscheinlichkeiten
        - Wahrscheinlichkeit, dass A besser ist als B (und umgekehrt)
        - Credible Interval der Differenz A − B
    """
    # Posterior-Parameter berechnen
    alpha_a = successes[0] + 1
    beta_a = trials[0] - successes[0] + 1
    alpha_b = successes[1] + 1
    beta_b = trials[1] - successes[1] + 1

    # Stichproben aus den Posterior-Verteilungen
    samples_a = beta.rvs(alpha_a, beta_a, size=n_samples)
    samples_b = beta.rvs(alpha_b, beta_b, size=n_samples)

    # Wahrscheinlichkeiten berechnen
    prob_a_better = np.mean(samples_a > samples_b)
    prob_b_better = np.mean(samples_b > samples_a)

    # Differenz und Credible Interval
    diff = samples_a - samples_b
    lower = float(np.percentile(diff, (1 - ci) / 2 * 100))
    upper = float(np.percentile(diff, (1 + ci) / 2 * 100))

    return {
        "Probability_A": np.mean(samples_a),
        "Probability_B": np.mean(samples_b),
        "Probability_A_Better": prob_a_better,
        "Probability_B_Better": prob_b_better,
        "Credible_Interval": (lower, upper)
    }

## Anwendungsbeispiel

In [9]:
# Beispiel
# A = 65 Erfolge, 90 Versuche
# B = 72 Erfolge, 110 Versuche
# Credible Interval = 95%

ci = 0.95
result = bayesian_ab_test([65, 72], [90, 110], ci)

print(f"Wahrscheinlichkeit, dass A erfolgreich ist: {result['Probability_A']:.3f}")
print(f"Wahrscheinlichkeit, dass B erfolgreich ist: {result['Probability_B']:.3f}")

print(f"Wahrscheinlichkeit, dass A besser ist als B: {result['Probability_A_Better']:.3f}")
print(f"Wahrscheinlichkeit, dass B besser ist als A: {result['Probability_B_Better']:.3f}")

print(f"Credible Interval ({ci*100})%: {result['Credible_Interval'][0]:.2f}, {result['Credible_Interval'][1]:.2f}")

Wahrscheinlichkeit, dass A erfolgreich ist: 0.717
Wahrscheinlichkeit, dass B erfolgreich ist: 0.652
Wahrscheinlichkeit, dass A besser ist als B: 0.845
Wahrscheinlichkeit, dass B besser ist als A: 0.155
Credible Interval (95.0)%: -0.06, 0.19


### 📊 Erklärung und Interpretation


#### Kontext der Analyse

* Variante **A**: 65 Erfolge bei 90 Versuchen → Erfolgsrate ≈ 72 %
* Variante **B**: 72 Erfolge bei 110 Versuchen → Erfolgsrate ≈ 65 %

Du möchtest wissen: **Ist A besser als B?**


#### Was sagt das Credible Interval?

```
Credible Interval (95%): –0.06, 0.19
```

➡️ Das bedeutet:
Mit **95 % Wahrscheinlichkeit** liegt der wahre Unterschied der Erfolgsraten zwischen **A und B** irgendwo zwischen **–6 Prozentpunkten und +19 Prozentpunkten**.

* Ein **positiver Wert** würde bedeuten: A ist besser als B
* Ein **negativer Wert** würde bedeuten: B ist besser als A


#### Interpretation im Beispiel

* Der Wert **0** liegt **innerhalb** des Intervalls → Wir können **nicht sicher** sagen, dass A **wirklich besser** ist.
* Die Wahrscheinlichkeit, dass A besser ist als B liegt zwar bei **84,8 %**, aber es gibt noch **Restunsicherheit**, da das Intervall auch **negative Unterschiede** zulässt.


#### 📌 Fazit

> A **ist wahrscheinlich besser**, aber der Unterschied ist **nicht 100 % sicher**.
> Das **Credible Interval zeigt, wie gross der Unterschied vermutlich ist** – inklusive Unsicherheit.

Das ist eine zentrale Stärke der **Bayesschen Analyse**: Sie zeigt dir **nicht nur, was wahrscheinlich ist**, sondern **wie sicher** du dir sein kannst.
